In [1]:
import time
import lattice_challenges_01

# in this notebook's comments, triangular matrix
#   should always mean lower triangular matrix

In [2]:

# in triangular matrix, this function changes row (vector)
#   at index line_index in input_matrix by subtracting row
#   at index by_index k times
def subtract_row_in_triangular(input_matrix, line_index, by_index, k):
    for i in range(by_index+1):
        input_matrix[line_index, i] -= k*input_matrix[by_index, i]

# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; the orthogonalized vector is v_i;
#   the vector by which it is improved is v_j
def gram_schmidt_mu(v_i, v_j):
    return (v_i*v_j)/(v_j*v_j)

# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; here, basis is the matrix of row-vectors
#   being orthogonalized and gso is the list (or matrix) of already
#   orthogonalized vectors
def mu(i, j, basis, gso):
    return gram_schmidt_mu(basis[i], gso[j])

# this function returns a matrix which is the result
#   of Gram-Shmidt orthogonalization process (not orthonormalization)
def gram_schmidt_orthogonalization(basis):
    ret = matrix(RR, basis.nrows(), basis.ncols())
    for i in range(basis.nrows()):
        vector_being_orthogonalized = basis[i]
        for j in range(i):
            vector_being_orthogonalized -= gram_schmidt_mu(vector_being_orthogonalized, ret[j])*ret[j]
        ret[i] = vector_being_orthogonalized
    return ret

# implementation of classical LLL algorithm
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (including 0.75)
def LLL_usual_condition(input_basis, delta):
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and (delta - mu(i, i-1, basis, gso)**2)*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    return basis

# implementation of LLL algorithm with different condition as used in
#   https://link.springer.com/chapter/10.1007/978-3-319-94821-8_10
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (not including 0.75)
def LLL_unusual_condition(input_basis, delta):
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    return basis

# auxiliary function used in triangularize_basis
def add_line_to_triangular_matrix_diag(ret, current_row, ort_basis, diag_value):
    for i in range(current_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[current_row], ort_basis[i])
        ort_basis[current_row] -= t*ort_basis[i]
        ret[current_row, i] = t*ret[i, i]
    ret[current_row, current_row] = diag_value

# auxiliary function used in triangularize_basis
def add_line_to_triangular_matrix(ret, current_row, ort_basis):
    for i in range(current_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[current_row], ort_basis[i])
        ort_basis[current_row] -= t*ort_basis[i]
        ret[current_row, i] = t*ret[i, i]
    ret[current_row, current_row] = sqrt(ort_basis[current_row]*ort_basis[current_row])

# function returning a triangular matrix which has the same Gramian matrix
#   as the input basis
# if number_of_small_diagonal_elements is zero,
#   it does not change the order of rows in basis
# if number_of_small_diagonal_elements is positive integer,
#   the ordering of rows in input basis is changed (side effect)
#   and the ordering of first number_of_small_diagonal_elements
#   rows is chosen (gradually, greedily) so that the corresponding
#   diagonal elaments in the resulting matrix are the lowest possible
def triangularize_basis(basis, number_of_small_diagonal_elements=0):
    ret = matrix(RR, basis.nrows(), basis.nrows())
    ort_basis = matrix(RR, basis)
    current_row = 0
    while current_row < number_of_small_diagonal_elements:
        min_so_far = math.inf  # using infinity from math module
        for m in range(current_row, ort_basis.nrows()):
            avec = ort_basis[m]
            for i in range(current_row):
                avec -= gram_schmidt_mu(avec, ort_basis[i])*ort_basis[i]
            t = sqrt(avec*avec)
            if t < min_so_far:
                min_so_far = t
                index_of_min_so_far = m
        if index_of_min_so_far != current_row:
            ort_basis[current_row], ort_basis[index_of_min_so_far] = ort_basis[index_of_min_so_far], ort_basis[current_row]
            basis[current_row], basis[index_of_min_so_far] = basis[index_of_min_so_far], basis[current_row]
        add_line_to_triangular_matrix_diag(ret, current_row, ort_basis, min_so_far)
        current_row += 1
    while current_row < ort_basis.nrows():
        add_line_to_triangular_matrix(ret, current_row, ort_basis)
        current_row += 1
    return ret

# this function reduces the input triangular_basis from bottom
# performs the same changes on parallel_basis
# the function has side effect on both input parameters
# the return value indicates whether any change of the input
#   parameters has been made (is True if so)
def reduce_triangular_basis(triangular_basis, parallel_basis):
    change = False
    for current_row in range(triangular_basis.nrows()-1, -1, -1):
        for m in range(current_row+1, triangular_basis.nrows()):
            t = round(triangular_basis[m, current_row]/triangular_basis[current_row, current_row])
            if t != 0:
                change = True
                subtract_row_in_triangular(triangular_basis, m, current_row, t)
                parallel_basis[m] -= t*parallel_basis[current_row]
    return change

# probably has exactly the same effect as reduce_triangular_basis above
def reduce_triangular_basis_from_top(triangular_basis, parallel_basis):
    change = False
    for m in range(1, triangular_basis.nrows()):
        for current_row in range(m-1, -1, -1):
            t = round(triangular_basis[m, current_row]/triangular_basis[current_row, current_row])
            if t != 0:
                change = True
                subtract_row_in_triangular(triangular_basis, m, current_row, t)
                parallel_basis[m] -= t*parallel_basis[current_row]
    return change

# old function for reduction (does not terminate in some cases)
# reduces basis using reduce_triangular_basis until something in the basis changes
# uses copy of input_basis (does not have side effects)
def reduce_basis_using_triangular(input_basis, number_of_small_diagonal_elements=0):
    basis = copy(input_basis)
    change = True
    while change:
        triangular_basis = triangularize_basis(basis, number_of_small_diagonal_elements)
        change = reduce_triangular_basis(triangular_basis, basis)
    return basis

# returns the sum of natural logarithms of the euclidean norms
#   of vectors of input_matrix
def euclidean_sum_of_logarithms_of_norms(input_matrix):
    sum_of_logarithms_of_squared_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = input_matrix[i]*input_matrix[i]
        sum_of_logarithms_of_squared_norms += float(log(t))
    return sum_of_logarithms_of_squared_norms/2

# returns the sum of natural logarithms of the elements of the input alist
def sum_of_logarithms(alist):
    ret = 0.0
    for val in alist:
        ret += float(log(val))
    return ret

# sorts input_matrix via side effect and returns a list of squared euclidean norms
def sort_by_norms(input_matrix):
    # should return (squared) norms
    squared_norms = [(input_matrix[i]*input_matrix[i], i) for i in range(input_matrix.nrows())]
    squared_norms.sort()
    perm_dict = {}
    for i in range(len(squared_norms)):
        perm_dict[i] = squared_norms[i][1]
    while perm_dict:
        start_index, next_index = perm_dict.popitem()
        start_row = input_matrix[start_index]
        cur_index = start_index
        while next_index != start_index:
            input_matrix[cur_index] = input_matrix[next_index]
            cur_index = next_index
            next_index = perm_dict.pop(next_index)
        input_matrix[cur_index] = start_row
    return [squared_norms[i][0] for i in range(input_matrix.nrows())]

# sort by euclidean norms and reduce until sum of logarithms gets lower
# works with a copy of input_basis, does not have side effects
def reduce_basis_using_triangular_new(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        triangular_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(triangular_basis, basis):
            return basis
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# performs changes in triangular matrix corresponding to swaping
#   rows at indices i and i-1 in (not triangular) basis, so that
#   gramian matrices of triangular_basis and basis remain the same
def swap_rows_in_triangular_basis(triangular_basis, i):
    bk = triangular_basis[i, i-1]
    ak = triangular_basis[i-1, i-1]
    bk1 = triangular_basis[i, i]
    triangular_basis[i, i-1] = sqrt(bk**2+bk1**2)
    triangular_basis[i-1, i-1] = ak*bk/triangular_basis[i, i-1]
    triangular_basis[i-1, i] = ak*bk1/triangular_basis[i, i-1]
    triangular_basis[i, i] = 0
    triangular_basis[i], triangular_basis[i-1] = triangular_basis[i-1], triangular_basis[i]
    for x in range(i+1, triangular_basis.nrows()):
        xa = triangular_basis[x, i-1]
        triangular_basis[x, i-1] = (xa*bk+triangular_basis[x, i]*bk1)/triangular_basis[i-1, i-1]
        triangular_basis[x, i] = (xa*ak-triangular_basis[x, i-1]*triangular_basis[i, i-1])/triangular_basis[i, i]

# faster implementation of classical LLL
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_usual_condition_using_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]**2*(delta-(triangular_basis[i, i-1]/triangular_basis[i-1, i-1])**2) > triangular_basis[i, i]**2:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

# faster implementation of LLL with different condition
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_unusual_condition_using_triangular(input_basis, delta):
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]*sqrt_delta > triangular_basis[i, i]:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis


In [3]:
# functions for running time measurement

def LLL_usual_condition_time(input_basis, delta):
    print("LLL_usual_condition_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and (delta - mu(i, i-1, basis, gso)**2)*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_usual_condition_time stops.")
    print()
    return basis

def LLL_unusual_condition_time(input_basis, delta):
    print("LLL_unusual_condition_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_unusual_condition_time stops.")
    print()
    return basis

def reduce_basis_using_triangular_time(input_basis, number_of_small_diagonal_elements):
    print("reduce_basis_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with number_of_small_diagonal_elements:")
    print(number_of_small_diagonal_elements)
    basis = copy(input_basis)
    start_time = time.time()
    change = True
    while change:
        triangular_basis = triangularize_basis(basis, number_of_small_diagonal_elements)
        change = reduce_triangular_basis(triangular_basis, basis)
    stop_time = time.time()
    print("measured times:")
    print((start_time, stop_time))
    print(stop_time - start_time)
    print("reduce_basis_using_triangular_time stops.")
    print()
    return basis

def reduce_basis_using_triangular_new_time(input_basis):
    print("reduce_basis_using_triangular_new_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    basis = copy(input_basis)
    start_time = time.time()
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    start_of_while_cycle_time = time.time()
    while True:
        previous = copy(basis)
        triangular_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(triangular_basis, basis):
            break
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            break
        previous_value = t
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("reduce_basis_using_triangular_new_time stops.")
    print()
    return previous

def LLL_usual_condition_using_triangular_time(input_basis, delta):
    print("LLL_usual_condition_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    number_of_vectors = input_basis.nrows()
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]**2*(delta-(triangular_basis[i, i-1]/triangular_basis[i-1, i-1])**2) > triangular_basis[i, i]**2:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_usual_condition_using_triangular_time stops.")
    print()
    return basis

def LLL_unusual_condition_using_triangular_time(input_basis, delta):
    print("LLL_unusual_condition_using_triangular_time has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    basis = copy(input_basis)
    start_time = time.time()
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    start_of_while_cycle_time = time.time()
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]*sqrt_delta > triangular_basis[i, i]:
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    stop_time = time.time()
    print("measured times:")
    print((start_time, start_of_while_cycle_time, stop_time))
    print((start_of_while_cycle_time - start_time, stop_time - start_time))
    print("LLL_unusual_condition_using_triangular_time stops.")
    print()
    return basis


In [4]:
# functions for measuring number of iterations or some operations

def LLL_usual_condition_iterations(input_basis, delta):
    print("LLL_usual_condition_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    while_iterations_counter = 0
    for_iterations_counter = 0
    swap_counter = 0
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        while_iterations_counter += 1
        for j in range(i-1, -1, -1):
            for_iterations_counter += 1
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and (delta - mu(i, i-1, basis, gso)**2)*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            swap_counter += 1
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    print((while_iterations_counter, for_iterations_counter, swap_counter))
    print("LLL_usual_condition_iterations has terminated at time:")
    print(time.time())
    print()
    return basis

def LLL_unusual_condition_iterations(input_basis, delta):
    print("LLL_unusual_condition_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    while_iterations_counter = 0
    for_iterations_counter = 0
    swap_counter = 0
    basis = copy(input_basis)
    number_of_vectors = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < number_of_vectors:
        while_iterations_counter += 1
        for j in range(i-1, -1, -1):
            for_iterations_counter += 1
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            swap_counter += 1
            g_i_1 = gso[i] + mu(i, i-1, basis, gso)*gso[i-1]
            gso[i] = gso[i-1] - gram_schmidt_mu(basis[i-1], g_i_1)*g_i_1
            gso[i-1] = g_i_1
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
        else:
            i += 1
    print((while_iterations_counter, for_iterations_counter, swap_counter))
    print("LLL_unusual_condition_iterations has terminated at time:")
    print(time.time())
    print()
    return basis

def reduce_basis_using_triangular_iterations(input_basis, number_of_small_diagonal_elements=0):
    print("reduce_basis_using_triangular_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with number_of_small_diagonal_elements:")
    print(number_of_small_diagonal_elements)
    while_iterations_counter = 0
    basis = copy(input_basis)
    change = True
    while change:
        while_iterations_counter += 1
        triangular_basis = triangularize_basis(basis, number_of_small_diagonal_elements)
        change = reduce_triangular_basis(triangular_basis, basis)
    print(while_iterations_counter)
    print("reduce_basis_using_triangular_iterations has terminated at time:")
    print(time.time())
    print()
    return basis

def reduce_basis_using_triangular_new_iterations(input_basis):
    print("reduce_basis_using_triangular_new_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    while_iterations_counter = 0
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        while_iterations_counter += 1
        previous = copy(basis)
        triangular_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(triangular_basis, basis):
            break
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            break
        previous_value = t
    print(while_iterations_counter)
    print("reduce_basis_using_triangular_new_iterations has terminated at time:")
    print(time.time())
    print()
    return previous

def LLL_usual_condition_using_triangular_iterations(input_basis, delta):
    print("LLL_usual_condition_using_triangular_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    while_iterations_counter = 0
    flag_true_counter = 0
    for_iterations_counter = 0
    swap_counter = 0
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        while_iterations_counter += 1
        if flag:
            flag_true_counter += 1
            for j in range(i-1, -1, -1):
                for_iterations_counter += 1
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]**2*(delta-(triangular_basis[i, i-1]/triangular_basis[i-1, i-1])**2) > triangular_basis[i, i]**2:
            swap_counter += 1
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    print((while_iterations_counter, for_iterations_counter, swap_counter, flag_true_counter))
    print("LLL_usual_condition_using_triangular_iterations has terminated at time:")
    print(time.time())
    print()
    return basis

def LLL_unusual_condition_using_triangular_iterations(input_basis, delta):
    print("LLL_unusual_condition_using_triangular_iterations has been launched at time:")
    print(time.time())
    print("on a matrix of dimension:")
    print(str(input_basis.nrows()) + " times " + str(input_basis.ncols()))
    print("with delta:")
    print(delta)
    while_iterations_counter = 0
    flag_true_counter = 0
    for_iterations_counter = 0
    swap_counter = 0
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    triangular_basis = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        while_iterations_counter += 1
        if flag:
            flag_true_counter += 1
            for j in range(i-1, -1, -1):
                for_iterations_counter += 1
                t = round(triangular_basis[i, j]/triangular_basis[j, j])
                subtract_row_in_triangular(triangular_basis, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and triangular_basis[i-1, i-1]*sqrt_delta > triangular_basis[i, i]:
            swap_counter += 1
            swap_rows_in_triangular_basis(triangular_basis, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    print((while_iterations_counter, for_iterations_counter, swap_counter, flag_true_counter))
    print("LLL_unusual_condition_using_triangular_iterations has terminated at time:")
    print(time.time())
    print()
    return basis


In [5]:
def taxicab_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t += abs(input_matrix[i, j])
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_norms)

def chebyshev_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t2 = abs(input_matrix[i, j])
            if t2 > t:
                t = t2
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_norms)

def euclidean_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0.0
    sum_of_norms = 0.0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0.0
        for j in range(input_matrix.ncols()):
            t += input_matrix[i, j]**2
        r = sqrt(t)
        sum_of_norms += r
        sum_of_logarithms_of_norms += float(log(r))
        if r > longest_vector_norm:
            longest_vector_norm = r
        if r < shortest_vector_norm:
            shortest_vector_norm = r
    return (shortest_vector_norm, longest_vector_norm, sum_of_norms, sum_of_logarithms_of_norms)

def all_properties(input_matrix):
    print(euclidean_properties(input_matrix))
    print(chebyshev_properties(input_matrix))
    print(taxicab_properties(input_matrix))


In [6]:
# assumed the inputs are integer matrix and integer vector of the same dimension (row length)
def check_vector_in_lattice(basis, a_vector):
    return all([basis.solve_left(a_vector).denominator() == 1])

# assumed the inputs are integer matrices (not necessarily) of the same dimension
def check_reduction_correctness(original_basis, tested_matrix):
    ret = True
    for a_row in tested_matrix:
        if not check_vector_in_lattice(original_basis, a_row):
            return False
    return original_basis.rank()==tested_matrix.rank()


In [7]:
def add_matrix_to_file(filename, matrixname, the_matrix):
    f = open(filename, 'a')
    f.write("\ndef "+str(matrixname)+"():\n    return [")
    for i in range(the_matrix.nrows()-1):
        f.write(str(list(the_matrix[i]))+",\n            ")
    if the_matrix.nrows()>0:
        f.write(str(list(the_matrix[the_matrix.nrows()-1])))
    f.write("]\n")
    f.close()


In [16]:
# aaa_archivovana_mereni/mereni_20201125a.py
for d in range(225,325,25):
    print("challenge "+str(d)+":")
    input_d = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    all_properties(input_d)
    print()
    print("Launching LLL_usual_condition with delta=0.75")
    
    start_time = time.time()
    output_d_LLL_usual_condition = LLL_usual_condition(input_d, 0.75)
    end_time = time.time()
    
    all_properties(output_d_LLL_usual_condition)
    print("Running time of LLL_usual_condition with delta=0.75 was: "+str(round(end_time-start_time, 6))+" s")
    print()
    print("Launching LLL_usual_condition_using_triangular with delta=0.75")
    
    start_time = time.time()
    output_d_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular(input_d, 0.75)
    end_time = time.time()
    
    all_properties(output_d_LLL_usual_condition_using_triangular)
    print("Running time of LLL_usual_condition_using_triangular with delta=0.75 was: "+str(round(end_time-start_time, 6))+" s")
    print()
    print("Launching reduce_basis_using_triangular_new")
    
    start_time = time.time()
    output_d_reduce_basis_using_triangular_new = reduce_basis_using_triangular_new(input_d)
    end_time = time.time()
    
    all_properties(output_d_reduce_basis_using_triangular_new)
    print("Running time of reduce_basis_using_triangular_new was: "+str(round(end_time-start_time, 6))+" s")
    print()
    print()
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201125a.py",
                       "LLL_usual_condition_on_"+str(d),
                       output_d_LLL_usual_condition)
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201125a.py",
                       "LLL_usual_condition_using_triangular_on_"+str(d),
                       output_d_LLL_usual_condition_using_triangular)
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201125a.py",
                       "reduce_basis_using_triangular_new_on_"+str(d),
                       output_d_reduce_basis_using_triangular_new)
    input_again = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    if not check_reduction_correctness(input_again, output_d_LLL_usual_condition):
        print("Wrongly reduced matrix output_d_LLL_usual_condition for d="+str(d))
    if not check_reduction_correctness(input_again, output_d_LLL_usual_condition_using_triangular):
        print("Wrongly reduced matrix output_d_LLL_usual_condition_using_triangular for d="+str(d))
    if not check_reduction_correctness(input_again, output_d_reduce_basis_using_triangular_new):
        print("Wrongly reduced matrix output_d_reduce_basis_using_triangular_new for d="+str(d))


challenge 225:
(33.0000000000000, 131.202134128984, 21719.5488315009, 1012.77060005235)
(27, 33, 7130, 777.491172992671)
(33, 702, 103756, 1320.47528501228)

Launching LLL_usual_condition with delta=0.75
(38.9743505398102, 77.2657750883274, 13307.8970613460, 916.483212749023)
(15, 42, 5452, 713.568502410249)
(235, 405, 72511, 1298.14596713489)
Running time of LLL_usual_condition with delta=0.75 was: 1097.539955 s

Launching LLL_usual_condition_using_triangular with delta=0.75
(38.9743505398102, 77.2657750883274, 13307.8970613460, 916.483212749023)
(15, 42, 5452, 713.568502410249)
(235, 405, 72511, 1298.14596713489)
Running time of LLL_usual_condition_using_triangular with delta=0.75 was: 318.180486 s

Launching reduce_basis_using_triangular_new
(33.0000000000000, 66.2268827591938, 11591.0925041949, 883.116659944259)
(13, 33, 4127, 645.508996328040)
(33, 348, 53680, 1192.05699682838)
Running time of reduce_basis_using_triangular_new was: 24.27477 s


challenge 250:
(36.0000000000000, 14

In [17]:
# aaa_archivovana_mereni/mereni_20201125b.py
for d in range(225,325,25):
    print("challenge "+str(d)+":")
    input_d = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    all_properties(input_d)
    print()
    print("Launching LLL from SageMath with delta=0.75")
    
    start_time = time.time()
    output_d_LLL_sage = input_d.LLL(delta=0.75)
    end_time = time.time()
    
    all_properties(output_d_LLL_sage)
    print("Running time of LLL from SageMath with delta=0.75 was: "+str(round(end_time-start_time, 6))+" s")
    print()
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201125b.py",
                       "LLL_from_SageMath_on_"+str(d),
                       output_d_LLL_sage)
    input_again = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    if not check_reduction_correctness(input_again, output_d_LLL_sage):
        print("Wrongly reduced matrix output_d_LLL_sage for d="+str(d))
    else:
        print("Checked.")
    print()


challenge 225:
(33.0000000000000, 131.202134128984, 21719.5488315009, 1012.77060005235)
(27, 33, 7130, 777.491172992671)
(33, 702, 103756, 1320.47528501228)

Launching LLL from SageMath with delta=0.75
(38.9743505398102, 77.2657750883274, 13314.8029625931, 916.599239389701)
(15, 42, 5462, 713.943195859690)
(235, 405, 72518, 1298.16794604161)
Running time of LLL from SageMath with delta=0.75 was: 32.271262 s

Checked.

challenge 250:
(36.0000000000000, 146.386474785070, 27353.0392489488, 1156.01624115161)
(30, 36, 8660, 886.144738180901)
(36, 800, 136341, 1507.76766700461)

Launching LLL from SageMath with delta=0.75
(54.5068802996466, 94.7997890293011, 18339.5529185497, 1072.34844210730)
(17, 52, 7239, 837.519156394279)
(297, 565, 102910, 1503.36216725484)
Running time of LLL from SageMath with delta=0.75 was: 39.306944 s

Checked.

challenge 275:
(38.0000000000000, 152.128235380550, 23777.4256838863, 1213.52601051350)
(19, 53, 9290, 964.762442473500)
(38, 821, 115430, 1601.95912970634

In [8]:
# aaa_archivovana_mereni/mereni_20201126a.py
for d in range(225,325,25):
    print("challenge "+str(d)+":")
    input_d = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    all_properties(input_d)
    print()
    print("Launching LLL_unusual_condition with delta=0.74")
    
    start_time = time.time()
    output_d_LLL_unusual_condition = LLL_unusual_condition(input_d, 0.74)
    end_time = time.time()
    
    all_properties(output_d_LLL_unusual_condition)
    print("Running time of LLL_unusual_condition with delta=0.74 was: "+str(round(end_time-start_time, 6))+" s")
    print()
    print("Launching LLL_unusual_condition_using_triangular with delta=0.74")
    
    start_time = time.time()
    output_d_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular(input_d, 0.74)
    end_time = time.time()
    
    all_properties(output_d_LLL_unusual_condition_using_triangular)
    print("Running time of LLL_unusual_condition_using_triangular with delta=0.74 was: "+str(round(end_time-start_time, 6))+" s")
    print()
    print()
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201126a.py",
                       "LLL_unusual_condition_on_"+str(d),
                       output_d_LLL_unusual_condition)
    add_matrix_to_file("aaa_archivovana_mereni/mereni_20201126a.py",
                       "LLL_unusual_condition_using_triangular_on_"+str(d),
                       output_d_LLL_unusual_condition_using_triangular)
    input_again = matrix(eval("lattice_challenges_01.challenge"+str(d)+"()"))
    if not check_reduction_correctness(input_again, output_d_LLL_unusual_condition):
        print("Wrongly reduced matrix output_d_LLL_unusual_condition for d="+str(d))
    if not check_reduction_correctness(input_again, output_d_LLL_unusual_condition_using_triangular):
        print("Wrongly reduced matrix output_d_LLL_unusual_condition_using_triangular for d="+str(d))


challenge 225:
(33.0000000000000, 131.202134128984, 21719.5488315009, 1012.77060005235)
(27, 33, 7130, 777.491172992671)
(33, 702, 103756, 1320.47528501228)

Launching LLL_unusual_condition with delta=0.74
(29.0516780926679, 59.1523456846810, 10412.3127370887, 861.606275507853)
(9, 28, 4003, 644.419504058296)
(206, 403, 66202, 1277.66938024418)
Running time of LLL_unusual_condition with delta=0.74 was: 1113.782124 s

Launching LLL_unusual_condition_using_triangular with delta=0.74
(29.0516780926679, 59.1523456846810, 10412.3127370887, 861.606275507853)
(9, 28, 4003, 644.419504058296)
(206, 403, 66202, 1277.66938024418)
Running time of LLL_unusual_condition_using_triangular with delta=0.74 was: 322.406116 s


challenge 250:
(36.0000000000000, 146.386474785070, 27353.0392489488, 1156.01624115161)
(30, 36, 8660, 886.144738180901)
(36, 800, 136341, 1507.76766700461)

Launching LLL_unusual_condition with delta=0.74
(38.3275357934736, 69.1230786351418, 13070.7933489277, 987.884271176452)
(11

In [24]:
print("challenge 225:")
input_225 = matrix(lattice_challenges_01.challenge225())
all_properties(input_225)
print()
output_225_LLL_usual_condition = LLL_usual_condition_iterations(input_225, 0.75)
all_properties(output_225_LLL_usual_condition)
print()
output_225_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular_iterations(input_225, 0.75)
all_properties(output_225_LLL_usual_condition_using_triangular)
print()
#output_225_LLL_unusual_condition = LLL_unusual_condition_iterations(input_225, 0.74)
#all_properties(output_225_LLL_unusual_condition)
#print()
#output_225_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular_iterations(input_225, 0.74)
#all_properties(output_225_LLL_unusual_condition_using_triangular)
#print()
output_225_reduce_basis_using_triangular_new = reduce_basis_using_triangular_new_iterations(input_225)
all_properties(output_225_reduce_basis_using_triangular_new)
print()
print()
print("challenge 250:")
input_250 = matrix(lattice_challenges_01.challenge250())
all_properties(input_250)
print()
output_250_LLL_usual_condition = LLL_usual_condition_iterations(input_250, 0.75)
all_properties(output_250_LLL_usual_condition)
print()
output_250_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular_iterations(input_250, 0.75)
all_properties(output_250_LLL_usual_condition_using_triangular)
print()
#output_250_LLL_unusual_condition = LLL_unusual_condition_iterations(input_250, 0.74)
#all_properties(output_250_LLL_unusual_condition)
#print()
#output_250_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular_iterations(input_250, 0.74)
#all_properties(output_250_LLL_unusual_condition_using_triangular)
#print()
output_250_reduce_basis_using_triangular_new = reduce_basis_using_triangular_new_iterations(input_250)
all_properties(output_250_reduce_basis_using_triangular_new)
print()
print()

print("challenge 275:")
input_275 = matrix(lattice_challenges_01.challenge275())
all_properties(input_275)
print()
output_275_LLL_usual_condition = LLL_usual_condition_iterations(input_275, 0.75)
all_properties(output_275_LLL_usual_condition)
print()
output_275_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular_iterations(input_275, 0.75)
all_properties(output_275_LLL_usual_condition_using_triangular)
print()
#output_275_LLL_unusual_condition = LLL_unusual_condition_iterations(input_275, 0.74)
#all_properties(output_275_LLL_unusual_condition)
#print()
#output_275_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular_iterations(input_275, 0.74)
#all_properties(output_275_LLL_unusual_condition_using_triangular)
#print()
output_275_reduce_basis_using_triangular_new = reduce_basis_using_triangular_new_iterations(input_275)
all_properties(output_275_reduce_basis_using_triangular_new)
print()
print()

print("challenge 300:")
input_300 = matrix(lattice_challenges_01.challenge300())
all_properties(input_300)
print()
output_300_LLL_usual_condition = LLL_usual_condition_iterations(input_300, 0.75)
all_properties(output_300_LLL_usual_condition)
print()
output_300_LLL_usual_condition_using_triangular = LLL_usual_condition_using_triangular_iterations(input_300, 0.75)
all_properties(output_300_LLL_usual_condition_using_triangular)
print()
#output_300_LLL_unusual_condition = LLL_unusual_condition_iterations(input_300, 0.74)
#all_properties(output_300_LLL_unusual_condition)
#print()
#output_300_LLL_unusual_condition_using_triangular = LLL_unusual_condition_using_triangular_iterations(input_300, 0.74)
#all_properties(output_300_LLL_unusual_condition_using_triangular)
#print()
output_300_reduce_basis_using_triangular_new = reduce_basis_using_triangular_new_iterations(input_300)
all_properties(output_300_reduce_basis_using_triangular_new)
print()
print()


challenge 225:
(33.0000000000000, 131.202134128984, 21719.5488315009, 1012.77060005235)
(27, 33, 7130, 777.491172992671)
(33, 702, 103756, 1320.47528501228)

LLL_usual_condition_iterations has been launched at time:
1605863846.5891438
on a matrix of dimension:
225 times 225
with delta:
0.750000000000000
(11287, 1362321, 5531)
LLL_usual_condition_iterations has terminated at time:
1605864925.6121593

(38.9743505398102, 77.2657750883274, 13307.8970613460, 916.483212749023)
(15, 42, 5452, 713.568502410249)
(235, 405, 72511, 1298.14596713489)

LLL_usual_condition_using_triangular_iterations has been launched at time:
1605864925.7462363
on a matrix of dimension:
225 times 225
with delta:
0.750000000000000
(11287, 696526, 5531, 5756)
LLL_usual_condition_using_triangular_iterations has terminated at time:
1605865242.2906446

(38.9743505398102, 77.2657750883274, 13307.8970613460, 916.483212749023)
(15, 42, 5452, 713.568502410249)
(235, 405, 72511, 1298.14596713489)

reduce_basis_using_triangul